In [5]:
import xarray as xr
import rioxarray
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.fill import fillnodata
from pathlib import Path

# ==== User settings ====
input_nc = r"C:\Users\Ankit\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc"
output_tif = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_30m_stack_filled_final.tif"
variables_to_process = ['t2m', 'd2m', 'u10', 'v10', 'tp']
target_resolution = 30  # meters
target_crs = "EPSG:32644"

# ---- STEP 1: Open dataset ----
ds = xr.open_dataset(input_nc)
print("Dataset dimensions:", ds.dims)
print("Variables:", list(ds.data_vars))

processed_bands = []

# ---- STEP 2: Process each ERA5 variable ----
for var in variables_to_process:
    print(f"Processing variable: {var}")
    da = ds[var]

    # Handle time dimension
    if "time" in da.dims:
        da = da.mean(dim="time")
    elif "valid_time" in da.dims:
        da = da.mean(dim="valid_time")

    # Attach CRS (ERA5 lat/lon WGS84)
    da = da.rio.write_crs("EPSG:4326")

    # Reproject to UTM & 30m
    da_utm = da.rio.reproject(
        target_crs,
        resolution=target_resolution,
        resampling=Resampling.bilinear
    )

    processed_bands.append(da_utm)

# ---- STEP 3: Stack into one multi-band raster ----
stacked = xr.concat(processed_bands, dim="band")
stacked = stacked.assign_coords(band=range(1, len(variables_to_process) + 1))

# ---- STEP 4: Convert to numpy & interpolate missing values ----
filled_bands = []
for i in range(len(variables_to_process)):
    arr = stacked.isel(band=i).values.astype("float32")

    # Mask nodata
    mask = np.isnan(arr)

    if mask.any():
        print(f"Interpolating nodata for band {i+1} ({variables_to_process[i]}) ...")
        arr_filled = fillnodata(arr, mask=mask, max_search_distance=50, smoothing_iterations=2)
    else:
        arr_filled = arr

    filled_bands.append(arr_filled)

filled_arr = np.stack(filled_bands, axis=0)

# ---- STEP 5: Build raster profile ----
transform = stacked.rio.transform()
height, width = stacked.rio.height, stacked.rio.width

profile = {
    "driver": "GTiff",
    "dtype": "float32",
    "count": len(variables_to_process),
    "crs": target_crs,
    "transform": transform,
    "height": height,
    "width": width,
    "compress": "LZW"
}

# ---- STEP 6: Save final GeoTIFF ----
with rasterio.open(output_tif, "w", **profile) as dst:
    dst.write(filled_arr)

print(f"✅ Final multi-band raster saved at: {output_tif}")

# ---- STEP 7: Quick sanity check ----
with rasterio.open(output_tif) as src:
    for i, var in enumerate(variables_to_process, 1):
        band = src.read(i)
        print(f"{var}: min={np.nanmin(band):.4f}, max={np.nanmax(band):.4f}")

Dataset dimensions: FrozenMappingWarningOnValuesAccess({'valid_time': 17544, 'latitude': 13, 'longitude': 17})
Variables: ['t2m', 'd2m', 'u10', 'v10', 'tp']
Processing variable: t2m
Processing variable: d2m
Processing variable: u10
Processing variable: v10
Processing variable: tp
Interpolating nodata for band 1 (t2m) ...
Interpolating nodata for band 2 (d2m) ...
Interpolating nodata for band 3 (u10) ...
Interpolating nodata for band 4 (v10) ...
Interpolating nodata for band 5 (tp) ...
✅ Final multi-band raster saved at: C:\Users\Ankit\Datasets_Forest_fire\ERA5_30m_stack_filled_final.tif
t2m: min=-9.2154, max=25.3370
d2m: min=-15.8976, max=19.1980
u10: min=-0.5856, max=1.5818
v10: min=-0.6061, max=2.1404
tp: min=0.0365, max=0.4218
